### Importação de Arquivos e Bibliotecas

In [ ]:
import pandas as pd
from pathlib import Path
import shutil
import os
import dataframe_image as dfi
import smtplib
import email.message

# Importa o arquivo externo com a senha do app para utilização do servidor google de envio de email
import senha_email

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
import threading


# Declaração das variáveis globais
META_FATURAMENTO_DIA = 1000
META_DIVERSIDADE_DIA = 4
META_TM_DIA = 500

META_FATURAMENTO_ANO = 1650000
META_DIVERSIDADE_ANO = 120
META_TM_ANO = 500

In [ ]:
# Importação das bases de dados de lojas, vendas e os emails de cada gerente de loja
lojas_tabela = pd.read_csv('Bases de Dados/Lojas.csv', encoding='ISO-8859-1', sep=';')
vendas_tabela = pd.read_excel('Bases de Dados/Vendas.xlsx')
emails_corrigidos = pd.read_csv('Bases de Dados/Emails.csv')

# Unifica a tabela de venndas com a lojas pelo ID da loja
vendas_lojas = pd.merge(vendas_tabela, lojas_tabela, on='ID Loja')

### Definindo funções

In [ ]:
# Retorna o dia do último registro de cada Loja.
def dia_atual(loja):
    dia_hoje = vendas_lojas[vendas_lojas['Loja'] == loja]
    dia_hoje = str(dia_hoje.iloc[-1,dia_hoje.columns.get_loc('Data')])
    return dia_hoje[:10]

In [ ]:
def enviar_email_massa(email_enviar, link_img_ano, link_img_dia, nome_gerente, dia, caminho_arquivo):

  """Envia um e-mail em massa com relatório de vendas, gráficos e planilha anexada.

  Esta função envia um e-mail para o gerente especificado, contendo um resumo
  do relatório de vendas do dia anterior, gráficos de desempenho diário e anual, 
  e a planilha detalhada como um anexo em formato CSV.

  Args:
    email_enviar: O endereço de e-mail do destinatário.
    link_img_ano: O caminho para o arquivo de imagem do gráfico de desempenho anual.
    link_img_dia: O caminho para o arquivo de imagem do gráfico de desempenho diário.
    nome_gerente: O nome do gerente para personalizar a saudação no e-mail. Com exceção da "Diretoria"
    dia: A data do relatório de vendas (usado no corpo do e-mail).
    caminho_arquivo: O caminho para o arquivo CSV contendo a planilha detalhada.

  Returns:
    None.

  Example:
    >>> enviar_email_massa(
    ...     'gerente@example.com',
    ...     '/caminho/para/grafico_anual.png',
    ...     '/caminho/para/grafico_diario.png',
    ...     'Carlos Silva',
    ...     '17/05',
    ...     '/caminho/para/relatorio_detalhado.csv'
    ... )
  """
    
    msg = MIMEMultipart()
    msg['Subject'] = "Relatório de vendas"
    msg['From'] = 'meuemail@gmail.com'
    msg['To'] = f'{email_enviar}'

    if nome_gerente != 'Diretoria':
        corpo_email = f"""
        <html>
        <head></head>
        <body>
            <p>Bom dia {nome_gerente},</p>
            <p>Segue o resultado do dia de ontem {dia}</p>
            <img src="cid:image_dia"><br><br>
            <img src="cid:image_ano"><br><br>
            <p>Segue em anexo a planilha detalhada</p>
        </body>
        </html>
        """
        msg.attach(MIMEText(corpo_email, "html"))
    
        # Anexar a imagem do dia
        with open(link_img_dia, 'rb') as img_dia_file:
            img_dia = MIMEImage(img_dia_file.read())
            img_dia.add_header('Content-ID', '<image_dia>')
            msg.attach(img_dia)
    
        # Anexar a imagem do ano
        with open(link_img_ano, 'rb') as img_ano_file:
            img_ano = MIMEImage(img_ano_file.read())
            img_ano.add_header('Content-ID', '<image_ano>')
            msg.attach(img_ano)
    
        # Anexar a planilha CSV
        with open(caminho_arquivo, 'rb') as planilha_file:
            attachment = MIMEApplication(planilha_file.read(), _subtype="csv")
            attachment['Content-Disposition'] = f'attachment; filename="{os.path.basename(caminho_arquivo)}"'
            msg.attach(attachment)

        servidor = smtplib.SMTP("smtp.gmail.com", 587)
        servidor.starttls()
        servidor.login(msg['From'], senha_email.senha_app)
        servidor.send_message(msg)
        servidor.quit()

In [ ]:
def email_nome_gerente(loja):

"""Retorna o e-mail e o nome do gerente de uma loja específica.

Busca no DataFrame 'emails_corrigidos' o e-mail e o nome do gerente
associados à loja fornecida.

Args:
  loja: O nome ou identificador da loja (string) para a qual se deseja
        obter o e-mail e o nome do gerente.

Returns:
  Uma tupla contendo dois strings:
     - O primeiro elemento é o endereço de e-mail do gerente.
     - O segundo elemento é o nome do gerente.
"""
    
    email = str(emails_corrigidos.loc[emails_corrigidos['Loja'] == loja, 'E-mail'].squeeze())
    nome = str(emails_corrigidos.loc[emails_corrigidos['Loja'] == loja, 'Gerente'].squeeze())
    return email, nome

In [ ]:
# Função auxiliar para exportar um DataFrame como img
def salvar_imagem(df, filename):
    try:
        dfi.export(df, filename)
    except Exception as e:
        print(f"Erro ao salvar a imagem {filename}: {e}")

### Criar as pastas de backup e adicionar as planilhas

In [ ]:
for loja in lojas_tabela['Loja']:
    dia_hoje = dia_atual(loja)

    # Cria as pastas de Backup para cada loja em caso de não existência
    pasta_destino = f'Backup Arquivos Lojas/{loja}/'
    os.makedirs(pasta_destino, exist_ok=True)

    # Salva a planilha Backup na pasta correspondente a loja
    vendas_lojas[vendas_lojas['Loja'] == loja].to_csv(f'{pasta_destino}/{loja} - {dia_hoje}.csv',index=False, encoding='utf-8')
    

### Criação dos indicadores e encaminhado por email

In [ ]:
for loja in lojas_tabela['Loja']:
    dia_hoje = dia_atual(loja)
    
    # Lê o arquivo CSV da loja para o dia atual.
    tabela_atual = pd.read_csv(f'Backup Arquivos Lojas/{loja}/{loja} - {dia_hoje}.csv')

    # Calcula o faturamento diário e anual.
    tabela_temp_indicadores = tabela_atual[tabela_atual['Data'] == dia_hoje]
    faturamento_dia = tabela_temp_indicadores['Valor Final'].sum()
    faturamento_ano = tabela_atual['Valor Final'].sum()

    # Calcula a diversidade de produtos vendidos no dia e no ano.
    diversidade = len(tabela_temp_indicadores.groupby('Produto').count())
    diversidade_ano = len(tabela_atual.groupby('Produto').count())

    # Calcula o ticket médio de vendas diário e anual.
    ticket_medio = round(tabela_temp_indicadores['Valor Final'].mean(),2)
    ticket_medio_ano = round(tabela_atual['Valor Final'].mean(),2)

    # Define os dados da tabela de indicadores diários.
    dados_tabela_dia = {
        'Indicador': ['Faturamento','Diversidade de Produtos','Ticket Médio por Venda'],
        'Valor Dia': [faturamento_dia, diversidade, ticket_medio],
        'Meta Dia': [META_FATURAMENTO_DIA, META_DIVERSIDADE_DIA, META_TM_DIA],
        'Cenário Dia': [
            '🟢' if faturamento_dia >= META_FATURAMENTO_DIA else '🔴',
            '🟢' if diversidade > META_DIVERSIDADE_DIA else '🔴',
            '🟢' if ticket_medio > META_TM_DIA else '🔴'
        ]
    }

    # Cria o DataFrame da tabela de indicadores diários.
    tabela_indicadores_dia = pd.DataFrame(dados_tabela_dia)
    tabela_indicadores_dia = tabela_indicadores_dia.set_index('Indicador')


    # Define os dados da tabela de indicadores anuais.
    dados_tabela_ano = {
        'Indicador': ['Faturamento','Diversidade de Produtos','Ticket Médio por Venda'],
        'Valor Ano': [faturamento_ano, diversidade_ano, ticket_medio_ano],
        'Meta Ano': [META_FATURAMENTO_ANO, META_DIVERSIDADE_ANO, META_TM_ANO],
        'Cenário Ano': [
            '🟢' if faturamento_ano >= META_FATURAMENTO_ANO else '🔴',
            '🟢' if diversidade_ano >= META_DIVERSIDADE_ANO else '🔴',
            '🟢' if ticket_medio_ano >= META_TM_ANO else '🔴'
        ]
    }

    # Cria o DataFrame da tabela de indicadores anuais.
    tabela_indicadores_ano = pd.DataFrame(dados_tabela_ano)
    tabela_indicadores_ano = tabela_indicadores_ano.set_index('Indicador')

    pasta_destino = f'Backup Arquivos Lojas/{loja}/'

    # Inicia thread para salvar imagem da tabela diária.
    nome_arquivo_dia = f'{pasta_destino}/{loja} - {dia_hoje}_dia.png'
    thread_dia = threading.Thread(target=salvar_imagem, args=(tabela_indicadores_dia, nome_arquivo_dia))
    thread_dia.start()

    # Inicia thread para salvar imagem da tabela anual.
    nome_arquivo_ano = f'{pasta_destino}/{loja} - {dia_hoje}_ano.png'
    thread_ano = threading.Thread(target=salvar_imagem, args=(tabela_indicadores_ano, nome_arquivo_ano))
    thread_ano.start()

    # Aguarda a conclusão das threads de salvamento de imagem.
    thread_dia.join()
    thread_ano.join()

    # Define o caminho do arquivo da planilha
    arquivo_planilha = f'{pasta_destino}/{loja} - {dia_hoje}.csv'

    # Obtém email/nome do gerente da loja
    email_gerente, nome_gerente = email_nome_gerente(loja)

    # Envia os relatórios
    enviar_email_massa(email_gerente, nome_arquivo_ano, nome_arquivo_dia, nome_gerente, dia_hoje[5:10], arquivo_planilha)

### Ranking para diretoria

In [ ]:
# Define o dia de referência como a data máxima no DataFrame.
dia_ref = vendas_lojas['Data'].max()

# Calcula e ordena o faturamento total por loja no ano.
faturamento_lojas_ano = vendas_lojas.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas_ano.sort_values(by ='Valor Final', ascending=False)

# Salva o ranking anual de faturamento em um arquivo CSV.
pasta_destino = f'Backup Arquivos Lojas/'
faturamento_lojas_ano.to_csv(f'{pasta_destino}/Rank Anual {dia_ref.day}-{dia_ref.month}.csv',index=False, encoding='utf-8')

#Calcula e ordena o faturamento por loja no dia de referência.
vendas_lojas_dia = vendas_lojas.loc[vendas_lojas['Data'] == dia_ref,:]
faturamento_lojas_dia = vendas_lojas_dia.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by ='Valor Final', ascending=False)

# Salva o ranking de faturamento do dia em um arquivo CSV.
faturamento_lojas_dia.to_csv(f'{pasta_destino}/Rank do Dia {dia_ref.day}-{dia_ref.month}.csv',index=False, encoding='utf-8')

### Envia o Ranking para diretoria

In [ ]:
# Define destinatário, assunto e remetente do e-mail.
email_enviar = emails_corrigidos.loc[emails_corrigidos['Loja']=='Diretoria', 'E-mail'].squeeze()

# Cria um objeto MIMEMultipart para construir o e-mail.
msg = MIMEMultipart()
msg['Subject'] = "Relatório de vendas"
msg['From'] = 'meuemail@gmail.com'
msg['To'] = f'{email_enviar}'

# Define o corpo do e-mail em formato HTML com ranking de lojas.
corpo_email = f"""
<html>
<head></head>
<body>
    <p>Prezados, Bom dia</p>
    <br>
    <p>Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${faturamento_lojas_dia.iloc[0,0]:.2f}</p>
    <p>Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${faturamento_lojas_dia.iloc[-1,0]:.2f}</p>
    <br>
    <p>Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${faturamento_lojas_ano.iloc[0,0]:.2f}</p>
    <p>Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R${faturamento_lojas_ano.iloc[-1,0]:.2f}</p>
    <br>
    <p>Segue em anexo o ranking do ano/dia de todas as lojas</p>
    <p>Qualquer dúvida estou a disposição</p>
</body>
</html>
"""
msg.attach(MIMEText(corpo_email, "html"))

# Anexa o arquivo CSV do ranking anual ao e-mail.
with open(f'Backup Arquivos Lojas/Rank Anual {dia_ref.day}-{dia_ref.month}.csv', 'rb') as planilha_file:
    attachment = MIMEApplication(planilha_file.read(), _subtype="csv")
    attachment['Content-Disposition'] = f'attachment; filename="{f'Rank Anual {dia_ref.day}-{dia_ref.month}'}"'
    msg.attach(attachment)

# Anexa o arquivo CSV do ranking diário ao e-mail.
with open(f'Backup Arquivos Lojas/Rank do Dia {dia_ref.day}-{dia_ref.month}.csv', 'rb') as planilha_file:
    attachment = MIMEApplication(planilha_file.read(), _subtype="csv")
    attachment['Content-Disposition'] = f'attachment; filename="{f'Rank do Dia {dia_ref.day}-{dia_ref.month}'}"'
    msg.attach(attachment)

# Conecta ao servidor SMTP do Gmail e envia o e-mail.
servidor = smtplib.SMTP("smtp.gmail.com", 587)
servidor.starttls()
servidor.login(msg['From'], senha_email.senha_app)
servidor.send_message(msg)
servidor.quit()